In [ ]:
import pandas as pd
import glob
import math 
import datetime
out_dir = "../data" #output directory
# read data from all files
result = pd.concat(
    [pd.read_csv(file, encoding="utf-8", parse_dates=[1,2,5], dayfirst=True) for file in glob.glob(out_dir + "/*.csv")],
    ignore_index=True)

In [ ]:
def get_summary(df_in, bins = [50, 250, 1000, 5000]):
    # count number of bets und sum
    df = df_in.groupby(by = ['author', 'status'], as_index=False).agg({'factor':['sum', 'count']})
    df.columns = ["_".join(x) for x in df.columns.ravel()]
    # make columns with detailed data on W-R-L
    df = df.pivot(index='author_', columns='status_', values=['factor_sum', 'factor_count'])
    df.columns = ["_".join(x) for x in df.columns.ravel()]
    df = df.fillna(value=0)
    # add summary columns
    df['count'] = df['factor_count_L'] + df['factor_count_R'] + df['factor_count_W']
    df['return'] = df['factor_sum_W'] + df['factor_count_R']
    df['win'] = df['factor_sum_W'] - df['factor_count_L'] - df['factor_count_W']
    df['success_rate'] = df['win'] / df['count'] * 100
    #df.sort_values(by=['success_rate'], ascending=False)
    #binned sorting
    df["exp"] = pd.cut(df['count'], bins)
    return df.sort_values(by=['exp','success_rate'], ascending=False)

In [ ]:
# alltime summary
get_summary(result)

In [ ]:
# only last year
get_summary(result[result.date >= datetime.date(2019, 1, 1)], [20, 100, 500, 2500])